In [ ]:
import potential_fitting

In [ ]:
H2O_settings = "H2O.ini"
ethanol_settings = "ethanol.ini"
dimer_settings = "dimer.ini"
unopt_H2O = "H2O.xyz"
unopt_ethanol = "ethanol.xyz"
opt_H2O = "H2O.opt.xyz"
opt_ethanol = "ethanol.opt.xyz"
training_configs = "training_configs.xyz"
test_configs = "test_configs.xyz"
database = "database.db"
training_set = "training_set.xyz"
test_set = "test_set.xyz"
poly_in = "poly.in"
molecule_in = "A1B2_C1D3E1F2G1H1"
poly_directory = "poly"
config = "config.ini"
ttm_directory = "ttm"
fit_directory = "fit"
fitted_code = "fitted"

In [ ]:
potential_fitting.optimize_geometry(H2O_settings, unopt_H2O, opt_H2O)
potential_fitting.optimize_geometry(ethanol_settings, unopt_ethanol, opt_ethanol)

In [ ]:
potential_fitting.generate_2b_configurations(dimer_settings, opt_H2O, opt_ethanol, 150, training_configs, min_distance = 2, max_distance = 10, min_inter_distance = 0)
potential_fitting.generate_2b_configurations(dimer_settings, opt_H2O, opt_ethanol, 150, test_configs, min_distance = 2, max_distance = 10, min_inter_distance = 0)

In [ ]:
potential_fitting.init_database(dimer_settings, database, training_configs, tag = "training")
potential_fitting.init_database(H2O_settings, database, opt_H2O, tag = "training")
potential_fitting.init_database(ethanol_settings, database, opt_ethanol, tag = "training")

potential_fitting.init_database(dimer_settings, database, test_configs, tag = "test")
potential_fitting.init_database(H2O_settings, database, opt_H2O, tag = "test")
potential_fitting.init_database(ethanol_settings, database, opt_ethanol, tag = "test")

In [ ]:
potential_fitting.fill_database(dimer_settings, database)

In [ ]:
potential_fitting.generate_2b_training_set(dimer_settings, database, training_set, "H2O", "ethanol", tag = "training")
potential_fitting.generate_2b_training_set(dimer_settings, database, test_set, "H2O", "ethanol", tag = "test")

In [ ]:
potential_fitting.generate_poly_input_from_database(dimer_settings, database, "H2O-ethanol", poly.in)

In [ ]:
potential_fitting.generate_polynomials(dimer_settings, poly_in, 1, poly_directory)

In [ ]:
potential_fitting.execute_maple(dimer_settings, poly_directory)

In [ ]:
potential_fitting.generate_fit_config(dimer_settings, molecule_in, config, opt_H2O, opt_ethanol)

In [ ]:
potential_fitting.generate_2b_ttm_fit_code(dimer_settings, config, molecule_in, ttm_directory)

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, ttm_directory)

In [ ]:
potential_fitting.fit_2b_ttm_training_set(dimer_settings, ttm_directory + "/fit-2b-ttm", training_set, ttm_directory, config)

In [ ]:
potential_fitting.generate_2b_fit_code(dimer_settings, config, poly_in, poly_directory, 1, fit_directory)

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, fit_directory)

In [ ]:
potential_fitting.fit_2b_training_set(dimer_settings, fit_directory + "/fit-2b", training_set, fit_directory, "fitted.nc")

In [ ]:
%matplotlib inline
potential_fitting.fitting.fit_visualizer.make_2b_graphs("ttm/eval-2b-ttm", "ttm/ttm-params.txt", "fit/eval-2b", "fitted.nc", database, "H2O", "ethanol", "%", "%", "%", "test")